In [1]:
%config ZMQInteractiveShell.ast_node_interactivity = "all"
%pprint

Pretty printing has been turned OFF


In [2]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [3]:
import sys
import torch
import torch.nn as nn
sys.path.append("../d2l_func/")
from data_prepare import load_data_jay_song, data_iter_random, data_iter_consecutive, to_onehot
from model_train import train_rnn, train_rnn_pytorch
from set_seed import set_seed

## GRU

当时间步过小或者过大时，RNN往往会出现梯度爆炸或者梯度消失的问题
- 梯度爆炸：可以通过梯度剪裁来解决
- 梯度消失：可以通过GRU/LSTM等来进行缓解

GRU包含重置门和更新门
- 重置门用于捕捉短期依赖关系
- 更新门用于捕捉长期依赖关系
- 重置门：$r_t = \delta(x_tw_{xr} + h_{t-1}w_{hr} + b_r)$
- 更新门：$z_t = \delta(x_tw_{xz} + h_{t-1}w_{hz} + b_z)$
- 候选隐藏层状态：$\widetilde h_t = tanh(x_tw_{xh} + r_t*(h_{t-1}w_hh) + b_h)$
- 当前时间步的隐藏层：$h_t = z*h_{t-1} + (1-z)\widetilde h_t$
- 输出层：$y_t = h_tw_{hy} + b_y$

### 自定义实现

#### 定义网络参数

In [4]:
def get_params(input_num, hidden_num, output_num, device):
    def _ones(shape):
        weight = nn.Parameter(torch.normal(0, 0.01, size=shape, device=device), requires_grad=True)
        return weight
    
    def _zeros(shape):
        bias = nn.Parameter(torch.zeros(shape, device=device), requires_grad=True)
        return bias
    
    def _three():
        return (
            _ones((input_num, hidden_num)),
            _ones((hidden_num, hidden_num)),
            _zeros(hidden_num),
        )
    
    w_xr, w_hr, b_r = _three()
    w_xz, w_hz, b_z = _three()
    w_xh, w_hh, b_h = _three()
    w_hy = _ones((hidden_num, output_num))
    b_y = _zeros((output_num))
    return nn.ParameterList([w_xr, w_hr, b_r, w_xz, w_hz, b_z, w_xh, w_hh, b_h, w_hy, b_y])

#### 定义gru层结构

In [5]:
from functools import reduce

def init_hidden_state(batch_size, hidden_num, device):
    return torch.zeros(batch_size, hidden_num, device=device)


def gru(inputs, h_state, params):
    w_xr, w_hr, b_r, w_xz, w_hz, b_z, w_xh, w_hh, b_h, w_hy, b_y = params
    outputs = []
    
    # inputs.shape is (num_step, batch_size, vocab_size)
    for x in inputs:
        rt = torch.sigmoid(torch.mm(x, w_xr) + torch.mm(h_state, w_hr) + b_r)
        zt = torch.sigmoid(torch.mm(x, w_xz) + torch.mm(h_state, w_hz) + b_z)
        h_candidate = torch.tanh(torch.mm(x, w_xh) + rt*torch.mm(h_state, w_hh) + b_h)
        h_state = zt*h_state + (1-zt)*h_candidate
        y = torch.mm(h_state, w_hy) + b_y
        outputs.append(y.unsqueeze(0))
        
    return reduce(lambda x, y: torch.cat((x, y)), outputs), h_state

In [6]:
# 验证
vocab_size, hidden_num = 15, 10
x = torch.arange(10).view(2, 5)
inputs = to_onehot(x, vocab_size, "cuda")
h_state = init_hidden_state(x.shape[0], hidden_num, "cuda")
params = get_params(vocab_size, hidden_num, vocab_size, "cuda")
outputs, h_state = gru(inputs, h_state, params)
outputs.shape, h_state.shape

(torch.Size([5, 2, 15]), torch.Size([2, 10]))

#### 字符级别预测

In [7]:
def predict_rnn(prefix, pred_num, model, init_hidden_state, hidden_num, 
                params, char_to_idx, vocab_set, vocab_size, device):
    outputs = [char_to_idx[prefix[0]]]
    h_state = init_hidden_state(1, hidden_num, device)
    
    for i in range(len(prefix) + pred_num - 1):
        # inputs.shape is (1, 1, vocab_size)
        inputs = to_onehot(torch.tensor(outputs[-1]).view(-1, 1), vocab_size, device)
        # y.shape is (1, 1, vocab_size), h_state.shape is (1, hidden_num)
        y, h_state = model(inputs, h_state, params)
        
        if i + 1 < len(prefix):
            outputs.append(char_to_idx[prefix[i+1]])
        else:
            outputs.append(y.argmax(dim=2).item())
            
    return "".join(vocab_set[i] for i in outputs)

In [8]:
# 验证
# load data
corpus_index, char_to_idx, vocab_set, vocab_size = load_data_jay_song()
# params
hidden_num, device = 256, "cuda"
params = get_params(vocab_size, hidden_num, vocab_size, device)
predict_rnn("分开", 10, gru, init_hidden_state, hidden_num, params, char_to_idx, vocab_set, vocab_size, device)

'分开宇墓2江伽逝星蹈哈哈'

#### 梯度剪裁

In [9]:
def grad_clipping(params, clipping_theta, device):
    # l2 norm
    norm = torch.zeros(1, device=device)
    # cumsum all grad data
    for param in params:
        norm += (param.grad.data ** 2).sum()
        
    norm = norm.sqrt()
    
    # grad explode
    if norm > clipping_theta:
        for param in params:
            param.grad.data *= (clipping_theta / norm) 

#### 训练

In [10]:
# training
import numpy as np
from sqdm import sqdm
from optim import sgd

def train_rnn(epoch_num, batch_num, model, loss, get_params, init_hidden_state, hidden_num, batch_size,
              lr, data_iter, prefixs, pred_num, corpus_index, char_to_idx, vocab_set, vocab_size, 
              num_step, predict_rnn, clipping_theta=1e-2, random_sample=True, device="cuda"):
    
    # init(use in calculate perplexity)
    l_sum, n_class = 0., 0.
    # training bar
    process_bar = sqdm()
    # init params
    params = get_params(vocab_size, hidden_num, vocab_size, device)
    
    for epoch in range(epoch_num):
        print(f"Epoch [{epoch+1}/{epoch_num}]")
        # sample in consecutive
        if not random_sample:
            h_state = init_hidden_state(batch_size, hidden_num, device)
        for x, y in data_iter(corpus_index, batch_size, num_step, device):
            # 原始x的shape为(batch_size, num_step)，onehot后的shape为(num_step, batch_size, vocab_size)
            x = to_onehot(x, vocab_size, device)
            if random_sample:
                h_state = init_hidden_state(x.shape[1], hidden_num, device)
            else:
                # 脱离计算图，使得上一时刻的隐藏状态变成叶子节点，防止在销毁计算图后（隐藏节点还存在），因反向传播到更早的
                # 隐藏层时刻（不在当前计算图内）而出错
                h_state.detach_()
                
            # model
            # outputs.shape is (num_step, batch_size, vocab_size), h_state.shape is (batch_size, hidden_num)
            outputs, h_state = model(x, h_state, params)
            # change output.shape --> (num_step, batch_size, vocab_size), 主要是为了方便计算loss
            outputs = outputs.view(-1, outputs.shape[-1])
            # 原始y的shape为(batch_size, num_step), ---> (num_step, batch_size) ---> 1维向量
            # 转置后变成内存不连续，使用contiguous变成连续的向量
            y = y.transpose(0, 1).contiguous().view(-1)
            
            # 计算loss, 标签需要是int
            l = loss(outputs, y.long())
            # grad clear
            if params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()
            # grad backward
            l.backward()
            # grad clipping
            grad_clipping(params, clipping_theta, device)
            # update grad
            sgd(params, lr)
            
            # calculate l_sum
            l_sum += l.item() * y.shape[0]
            n_class += y.shape[0]
            
            # calculate perplexity
            try:
                perplexity = np.exp(l_sum / n_class)
            except OverflowError:
                perplexity = float("inf")
                
            # training bar
            process_bar.show_process(batch_num, 1, perplexity)
        print("\n")
        # predict
        for prefix in prefixs:
            print(f"prefix-{prefix}: ", predict_rnn(prefix, pred_num, model, init_hidden_state, hidden_num, 
                                                    params, char_to_idx, vocab_set, vocab_size, device))
        print("\n")

In [11]:
# load data
corpus_index, char_to_idx, vocab_set, vocab_size = load_data_jay_song()

super_params = {
        "epoch_num": 250,
        "model": gru,
        "loss": nn.CrossEntropyLoss(),
        "init_hidden_state": init_hidden_state,
        "hidden_num": 256,
        "get_params": get_params,
        "batch_size": 64,
        "num_step": 32,
        "corpus_index": corpus_index,
        "data_iter": data_iter_random,
        "lr": 100,
        "char_to_idx": char_to_idx,
        "vocab_set": vocab_set,
        "vocab_size": vocab_size,
        "predict_rnn": predict_rnn,
        "pred_num": 50,
        "prefixs": ["分开", "不分开"],
        #     "random_sample": False
    }

super_params["batch_num"] = len(list(data_iter_random(corpus_index, super_params["batch_size"],
                                                      super_params["num_step"], "cpu")))

train_rnn(**super_params)

Epoch [1/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 856.6398, train_score: -, test_loss: -, test_score: --

prefix-分开:  分开                                                  
prefix-不分开:  不分开                                                  


Epoch [2/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 683.0150, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的
prefix-不分开:  不分开的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的


Epoch [3/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 614.0556, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开                                                  
prefix-不分开:  不分开                                                  


Epoch [4/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 573.0361, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我
prefix-不分开:  不分开我我我我我我我我我我我我我我我我我我我我我我我我我我我我

31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 326.9749, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开 我不能再想 我不能 我不能 我不能 我不能 我不能 我不能 我不能 我不能 我不能 我不能 我不能
prefix-不分开:  不分开 我不能再不要 我不能再不要 我不能再不要 我不能再不要 我不能再不要 我不能再不要 我不能再不要 


Epoch [34/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 321.4425, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开 我不能不能 我不能 我不能 我不能 我不能 我不能 我不能 我不能 我不能 我不能 我不能 我不能
prefix-不分开:  不分开 我不能不能 我不能 我不能 我不能 我不能 我不能 我不能 我不能 我不能 我不能 我不能 我不能


Epoch [35/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 316.0087, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开 我不能 我                                            
prefix-不分开:  不分开 我不能 我                                            


Epoch [36/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 310.5876, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开 我们的爱 你的爱 我不能 我不能 我不能 我不能 我不能 我不能 我不能 我不能 我不能 我不能 
prefix-不分开:  不分开 我不能再不能 我不能不能 我不能 你不能 我不能 我不能 我不能 我不能 我不

31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 178.4779, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开始不了我 我我我 我我我 我我我 我我我 我我我 我我我 我我我 我我我 我我我 我我我 我我我 我
prefix-不分开:  不分开 我不能再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我


Epoch [66/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 174.8754, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开始不能 我们的爱情 就让我们在演服 你说你不懂 你的笑变已经着我 我不能再想 我不要再想 我不要再想
prefix-不分开:  不分开                                                  


Epoch [67/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 171.3505, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开始不了 我用一步一步三步四步青春 在我的怀里你不用 我轻轻地尝一口 你说的爱我 不知不觉 我的解 这
prefix-不分开:  不分开 我不能再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我


Epoch [68/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 167.8342, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开 我没有你的手 只想你说我的爱情 你说了爱我 说你是我的选你 你说了这样的我知道          
prefix-不分开:  不分开 我不能再想 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 

31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 92.9371, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开始不出 你我的过去 被小时外的地边 口持成了一道 我们的感觉 你看着我 说你有些雨 一起上 的感觉 
prefix-不分开:  不分开 我要一定一步往上爬 在等待 被小时外的溪头 口旧的风 在等待雨来 我想起 你们的脸子       


Epoch [98/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 91.1583, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开始不出来 你说我怎么睡　 我不要再想 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我
prefix-不分开:  不分开 我不能再想 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我


Epoch [99/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 89.3874, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开始不能说你 我会发著呆   这个世界  有什么叫别  你叫你的想念 你的微笑 不用承导 我的解 这里
prefix-不分开:  不分开 我不能再这样打我妈妈 我说你的你说过去的温柔 我轻轻地尝一口你说的爱我 还在回味你给过的温柔 我轻


Epoch [100/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 87.6402, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开始不出来 你说你怎么借我 我不能再想 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我
prefix-不分开:  不分开 我不能再想 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 

31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 52.5471, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开始不出来 我不要再想 我不 我不 我不要再想你 不知不觉 你已经离开我 不知不觉 我跟了这节奏 后知
prefix-不分开:  不分开 我在等 一堆 强练不停 为了几家 继续着白 飞到一件落在飘移 是我的秘气 没让它受它的相爱 我说你


Epoch [130/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 51.7063, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开始不出 你说我的眼泪 让它留在雨天 哦 越过你划的线 我定了勇气的终点 你说我爱你 可爱我 相爱 我
prefix-不分开:  不分开 我不要再想 我不 我不 我不要再想你 不知不觉 你已经离开我 不知不觉 我跟了这节奏 后知后觉 又


Epoch [131/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 50.8784, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开始不出来 这样的甜蜜 让我开始乡相相命运 感觉到底探了来好 爱才有回到你知道 但是偏不了口让我知道 
prefix-不分开:  不分开 我不要再想 我不 我不 我不要再想你 不知不觉 你已经离开我 不知不觉 我跟了这节奏 后知后觉 后


Epoch [132/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 50.0889, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开始不出来 我不要再想 我不 我不 我不要再想你 不知不觉 你已经离开我 不知不觉 我跟了这节奏 后知
prefix-不分开:  不分开 我要一定一步往上爬 等着阳光 不需要用崇拜 我不是因为会比会走到我 我用第一人称

31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 33.0176, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开始不出来 我不要再想 我不要再想 我不 我不 我不要再想你 不知不觉 你已经离开我 不知不觉 我跟了
prefix-不分开:  不分开 我不能就这样失去你的微笑 口溢烟琴的让我们感会感动的可以 你是我的眼泪  叫他不得听 忘记 你的声


Epoch [162/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 32.5913, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开始不出来 你说你若便 已无法再重上谁 风在山路吹 过往的画面 全都是我不对 细数惭愧我伤你几回 我一
prefix-不分开:  不分开 我要一定一步往上爬 在最高点乘着叶片往前飞 小小的天流过的道和我 漂亮的距离 听你的笑我 还在回味


Epoch [163/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 32.1760, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开始不出 你我把爱过去 温暖了过去 我等着一点 你们的爱你 傻暖了吗 我的感觉 你们听着了手 说知不觉
prefix-不分开:  不分开 我不能再想 我不能再想 我不 我不 我不要再想你 不知不觉 你已经离开我 不知不觉 我跟了这节奏 


Epoch [164/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 31.7636, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开始不出 一直都好好 谁都有舍不着你 我知道做自己　别人是故事光 对着这张海报　我们在远距离欣赏与微笑
prefix-不分开:  不分开 我不能就这样失去你的微笑 口预烟回想你 是你说的玩生 如何说 看不到 从远方的酒

31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 22.6563, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开始不知道不会 我不能再想 我不 我不 我不要再想你 爱情来的太快就像龙卷风 离不开暴风圈来不及逃 我
prefix-不分开:  不分开 我不能再想 我不要再想 我不要再想 我不 我不 我不要再想你 爱情来的太快就像龙卷风 离不开暴风圈


Epoch [194/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 22.4144, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开始不出 这里都不得得 就要这样怎么抱　 你说不该再相见只为了瞬间 谢谢你让我听见 因为我在等待永远 
prefix-不分开:  不分开 我不能再想 我不能再想 我不 我不 我不要再想你 不知不觉 你已经离开我 不知不觉 我跟了这节奏 


Epoch [195/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 22.1782, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开始不停是一天 如果说了太多 也许颓子都想分手牵你不回 来不及易笑 如果这种海力　算自己 他们猜看我猜
prefix-不分开:  不分开 我不能再这样打着你的你 不知不来 你已经离开我 不知不觉 你跟王心 谁生小暴不 走得很分究 你说你


Epoch [196/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 21.9463, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开始不出 我说好好一路 一句两种三步四步望著天 看星星 一颗两颗三颗四颗 连成线背著背默默许下心愿 看
prefix-不分开:  不分开 我在等 一堆序 后再开 不稀 你看很难默 我知道不能再留 你永远赢不了 我永远做

31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 16.6235, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开始打手 远方的风车 远距离诉说 那幸福在深秋 满满的被收割 老仓库的角落 我们数着 一麻袋的爱跟快乐
prefix-不分开:  不分开 我不能就这样失去你的微笑 口红待在桌脚 而你我找不到 若角色对调 你说好不好 你的笑 你的好 脑海


Epoch [226/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 16.4799, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开始打出 爱你穿越过去 那么多余就怎么期　 却记得你说你发现 想象到你飞家电宙 面对海风蓝      
prefix-不分开:  不分开 我不想就这样牵着你的手不放开 爱可不可以简简单单没有伤害 你 靠着我的肩膀 你 在我胸口睡著 像这


Epoch [227/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 16.3373, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开始打出 爱你穿越时间 两行来自秋末的眼泪 让爱渗透了地面 我要的只是你在我身边 轻轻的叹息  后悔着
prefix-不分开:  不分开 我在等待 画面序 我辈子 再来一口 拯救了 清晰的 一切 时光机 你我翻滚 的小巷 背下一台 在感


Epoch [228/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 16.1948, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开始不停 一角的时候 整头也还在等待 以为你还会回来 你的脸慢慢离开 时间快将我掩埋 消失得太快 我负
prefix-不分开:  不分开 我不用豆腐e下的夜 在我地盘这   你就得听我的 节奏在招惹   我跟街舞亲热 

### 简单实现

#### 网络定义

In [16]:
class RNNModel(nn.Module):
    def __init__(self, rnn_layer, vocab_size):
        super(RNNModel, self).__init__()
        self.rnn = rnn_layer
        self.hidden_num = self.rnn.hidden_size * (2 if self.rnn.bidirectional else 1)
        self.vocab_size = vocab_size
        self.fc = nn.Linear(hidden_num, vocab_size)
        self.h_state = None
        
    def forward(self, x, h_state):
        # x.shape is (num_step, batch_size, vocab_size)
        y, self.h_state = self.rnn(x, h_state)
        return self.fc(y), self.h_state

#### 预测

In [17]:
def predict_rnn_pytorch(prefix, pred_num, model, char_to_idx, vocab_set, vocab_size, device):
    outputs = [char_to_idx[prefix[0]]]
    h_state = None
    
    for i in range(len(prefix) + pred_num - 1):
        inputs = to_onehot(torch.tensor(outputs[-1]).view(-1, 1), vocab_size, device)
        if h_state is not None:
            if isinstance(h_state, tuple): # lstm , (h,c)
                h_state = (h_state[0].to(device), h_state[1].to(device))
            else:
                h_state = h_state.to(device)
                
        y, h_state = model(inputs, h_state)
        if i + 1 < len(prefix):
            outputs.append(char_to_idx[prefix[i+1]])
        else:
            outputs.append(y.argmax(dim=2).item())
            
    return "".join(vocab_set[i] for i in outputs)

In [18]:
# 验证
# load data
hidden_num = 256
corpus_index, char_to_idx, vocab_set, vocab_size = load_data_jay_song()
rnn_layer = nn.GRU(vocab_size, hidden_num)
model = RNNModel(rnn_layer, vocab_size)
model = model.cuda()
predict_rnn_pytorch("分开", 10, model, char_to_idx, vocab_set, vocab_size, "cuda")

'分开稳睫奇跑跑得睫跑跑跨'

#### 训练

In [19]:
# training
import numpy as np
from sqdm import sqdm
from optim import sgd

def train_rnn_pytorch(epoch_num, batch_num, model, loss, optimizer, batch_size, data_iter, prefixs, 
                      pred_num, corpus_index, char_to_idx, vocab_set, vocab_size, num_step, predict_rnn_pytorch, 
                      clipping_theta=1e-2, random_sample=True, device="cuda"):
    
    # init(use in calculate perplexity)
    l_sum, n_class = 0., 0.
    # training bar
    process_bar = sqdm()
    
    for epoch in range(epoch_num):
        print(f"Epoch [{epoch+1}/{epoch_num}]")
        # sample in consecutive
        if not random_sample:
            h_state = None
        for x, y in data_iter(corpus_index, batch_size, num_step, device):
            # 原始x的shape为(batch_size, num_step)，onehot后的shape为(num_step, batch_size, vocab_size)
            x = to_onehot(x, vocab_size, device)
            if random_sample:
                h_state = None
            else:
                # 脱离计算图，使得上一时刻的隐藏状态变成叶子节点，防止在销毁计算图后（隐藏节点还存在），因反向传播到更早的
                # 隐藏层时刻（不在当前计算图内）而出错
                if h_state is not None:
                    if isinstance(h_state, tuple): # lstm, state: (h, c)
                        h_state = (h_state[0].detach(), h_state[1].detach())
                    else:
                        h_state.detach_()
                
            # model
            # outputs.shape is (num_step, batch_size, vocab_size), h_state.shape is (batch_size, hidden_num)
            outputs, h_state = model(x, h_state)
            # change output.shape --> (num_step, batch_size, vocab_size), 主要是为了方便计算loss
            outputs = outputs.view(-1, outputs.shape[-1])
            # 原始y的shape为(batch_size, num_step), ---> (num_step, batch_size) ---> 1维向量
            # 转置后变成内存不连续，使用contiguous变成连续的向量
            y = y.transpose(0, 1).contiguous().view(-1)
            
            # 计算loss, 标签需要是int
            l = loss(outputs, y.long())
            # grad clear
            optimizer.zero_grad()
            # grad backward
            l.backward()
            # grad clipping
            grad_clipping(model.parameters(), clipping_theta, device)
            # update grad
            optimizer.step()
            
            # calculate l_sum
            l_sum += l.item() * y.shape[0]
            n_class += y.shape[0]
            
            # calculate perplexity
            try:
                perplexity = np.exp(l_sum / n_class)
            except OverflowError:
                perplexity = float("inf")
                
            # training bar
            process_bar.show_process(batch_num, 1, perplexity)
        print("\n")
        # predict
        for prefix in prefixs:
            print(f"prefix-{prefix}: ", predict_rnn_pytorch(prefix, pred_num, model, char_to_idx, vocab_set, vocab_size, device))
        print("\n")

In [20]:
hidden_num = 256
rnn_layer = nn.GRU(vocab_size, hidden_num)
model = RNNModel(rnn_layer, vocab_size)
model = model.cuda()
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

params = {
    "epoch_num": 250,
    "model": model,
    "loss": loss,
    "optimizer": optimizer,
    "batch_size": 64,
    "num_step": 32,
    "corpus_index": corpus_index,
    "data_iter": data_iter_random,
    "char_to_idx": char_to_idx,
    "vocab_set": vocab_set,
    "vocab_size": vocab_size,
    "predict_rnn_pytorch": predict_rnn_pytorch,
    "pred_num": 50,
    "prefixs": ["分开", "不分开"],
#     "random_sample": False
}

params["batch_num"] = len(list(data_iter_random(corpus_index, params["batch_size"],
                                                     params["num_step"], "cpu")))

train_rnn_pytorch(**params)

Epoch [1/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 923.6031, train_score: -, test_loss: -, test_score: --

prefix-分开:  分开                                                  
prefix-不分开:  不分开                                                  


Epoch [2/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 640.9834, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开                                                  
prefix-不分开:  不分开                                                  


Epoch [3/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 556.9185, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开                                                  
prefix-不分开:  不分开                                                  


Epoch [4/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 512.9613, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开                                                  
prefix-不分开:  不分开                            

31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 100.5102, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开 我不能再想 我不要再想 你说不到  我不能再想 我不要再想 你说不到  我不能再想 我不要再想 你
prefix-不分开:  不分开 我不能再想 我不要再想 你说不到  我不能再想 我不要再想 你说不到  我不能再想 我不要再想 你


Epoch [34/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 95.9812, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开 我不能再想 我不能再想 我不能再想 我不能再想 我不能再想 我不能再想 我不能再想 我不能再想 我
prefix-不分开:  不分开 我不能再想 我不能再想 我不能再想 我不能再想 我不能再想 我不能再想 我不能再想 我不能再想 我


Epoch [35/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 91.7389, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开 我不能再想 我不能再想 我不能再想 我不能再想 我不能再想 我不能再想 我不能再想 我不能再想 我
prefix-不分开:  不分开 我不能再想 我不能再想 我不能再想 我不能再想 我不能再想 我不能再想 我不能再想 我不能再想 我


Epoch [36/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 87.7533, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开 我不能再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我
prefix-不分开:  不分开 一定一直在 我们的感觉 我不能再想 我不要再想 我不要再想 我不要再想 我不要再想 

31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 30.6639, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开始重出手 一阵莫名和利都不拿 它在灌木丛旁邂逅 一只令它心仪的母斑鸠 我的眼光　我看见你的泪水  一
prefix-不分开:  不分开 为什么我有你的样子 你说你不想要的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 坏


Epoch [66/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 29.7567, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开始重出手  原来爱跟我的味道这 这样子 你是否还在 琴声何来 生死难猜 用一起叫做家 没有了证明 没
prefix-不分开:  不分开 爱上 是因为我在等 我只是我不能再想 我不 再想 我不了 爱你了 不用麻烦了 不用麻烦了 不用麻烦


Epoch [67/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 28.9198, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开始重来　 我们一半 美艳的符号 来找我 别怪我 想要你想念 你说的好一点 从来不及逃 我不能再想 我
prefix-不分开:  不分开 爱上 是因为我在等 我只想这个奖  不要再想 我不  不 我不想 你发如雪 凄美了离别 我 你发脾


Epoch [68/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 28.1110, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开 为什么这样子 你说啊 你说 我不能再想 我不  不 我不要再想 我不 我不 我不要再想 我不 我不
prefix-不分开:  不分开 爱上 是因为我在等 我只是谁在练我 等待救援 我拉着我 想要你想要的想念 你说的爱我 

31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 14.2558, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开始乡相信命运 感谢地心引力 让我碰到你 漂亮的让我面红的可爱女人 温柔的让我心疼的可爱女人 透明的让
prefix-不分开:  不分开 说不要再想 我不 我不 我不要再想 我不 我不 我不要再想 我不 我不 我不要再想 我不 我不 你


Epoch [98/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 13.9882, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开始乡相信命运 感谢地心引力 让我碰到你 漂亮的让我面红的可爱女人 温柔的让我心疼的可爱女人 透明的让
prefix-不分开:  不分开 是因为我想要你说好 说你的爱我 还记得你说家是唯一的城堡 随着稻香河流继续奔跑 微微笑 小时候的梦


Epoch [99/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 13.7317, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开始乡相信命运 感谢地心引力 让我碰到你 漂亮的让我面红的可爱女人 温柔的让我心疼的可爱女人 透明的让
prefix-不分开:  不分开 是因为我想要 你已经不了我 你过我的情节 要爱你 我用手牵手 你抬头  说你不该怎么会扯去 你的事


Epoch [100/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 13.4814, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开不知道 你没有舍不得 你说你也会不会感到更加沮丧   难道这不是我要的天堂景象 沉沦假象 你只会感到
prefix-不分开:  不分开 爱上 是谁在窗外面徘徊 是我错失的机会 你站的方位 跟我中间隔着泪 街景一直在后退 

31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 8.6436, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开不知道 你会把你的灵魂  活 说好有些事太多 带着我说太快长大 才能保护她 美丽的白发 幸福中发芽 
prefix-不分开:  不分开 爱上 是谁说没有错 只是放手会比较好过 最美的爱情 回忆里待续 我命格无双 一统江山 狂胜之中 我


Epoch [130/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 8.5345, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开不知道 你会不会懂事你不了 不要再这样打我妈妈 我说的风 你无声再变 我已经离不掉 你在我 等 我 
prefix-不分开:  不分开 爱上 是谁来没有你的我 我轻轻地尝一口 你说的爱我 还在回味你给过的温柔 我轻轻地尝一口 这香浓的


Epoch [131/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 8.4315, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开不知道 你会不会懂不懂事做 我也离不能再牵手不去 我去了吧 我知道你不会怎么会 你说不出手 海鸟跟鱼
prefix-不分开:  不分开 没有爱过的 清楚了 我爱  麦搁一个人咧生气 乎伊烦恼 乎伊操心 虽然不关我的代誌 谁叫他是我的兄


Epoch [132/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 8.3261, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开不知道 你会更难过 你发如雪 凄美了离别 我焚香感动了谁 邀明月 让回忆皎洁 爱在月光下完美 你发如
prefix-不分开:  不分开 没有你 我不想 我不再一个人手 在我胸口睡著 像这样的生活 我爱你 你爱我 我不是你爸 

31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 6.1253, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开不知道 你说你好累 已经决定不走错了 你好笑 你说的爱我而过 请你记得我 跑得比别 我才有个依靠 有
prefix-不分开:  不分开 没办法 我不需要解释 所以他小丑我是大师 你的回话凌乱着 在这个时刻 我想起喷泉旁的白鸽  甜蜜散


Epoch [162/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 6.0700, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开不知道 你会不会学会尖叫  尖叫  比从前度何必听我吠 再不走有今生无下世 你是否想我起这个毒誓 宁
prefix-不分开:  不分开不知道 就算没有结果 我也能够承受 我知道你的痛 是我给的承诺 你说给过我纵容 沉默是因为包容 如果


Epoch [163/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 6.0166, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开不知道 你会更难过 你已经远远离开 我也会慢慢走开 为什么我连分开都迁就着你 我真的没有天份 安静的
prefix-不分开:  不分开 没办法 我不需要解释 所以他小丑我是大师 你的回话凌乱着 在这个时刻 我想起喷泉旁的白鸽  甜蜜散


Epoch [164/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 5.9641, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开不知道 你会更难过 你已经远远离开 我也会慢慢走开 为什么我连分开都迁就着你 我真的没有天份 安静的
prefix-不分开:  不分开 爱深埋珊瑚海  周杰伦  毁坏的沙雕如何重来 有裂痕的爱怎么重盖 只是一切都想要 我感动

31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 4.7887, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开不知道 你会不会懂不见罪的糖果 不知不觉 你已经离开我 不知不觉 我跟了这节奏 后知后觉 又过了一个
prefix-不分开:  不分开 没有你 我不需要我这个吧 又想了谁都难过 印地安斑鸠 会学人开口 仙人掌怕羞 蜥蝪横著走 这里什么


Epoch [194/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 4.7571, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开不知道 你说你好累 已无法再爱上谁 风在山路吹 过往的画面 全都是我不对 细数惭愧我伤你几回 我一路
prefix-不分开:  不分开 没有你 我不需要我这个光  你回家看看见 上色的美丽 没有了雨 雨刷开始不停左右 就像回忆 开始对


Epoch [195/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 4.7264, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开不知道 你会不会一直走到最后 就算没有结果 我也能够承受 我知道你的痛 是我给的承诺 你说给过我纵容
prefix-不分开:  不分开 没有你 我不需要被崇拜 我不需要被崇拜 我跨越过时代 如兽般的姿态 琴声唤醒沈睡的血脉 不需要被崇


Epoch [196/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 4.6958, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开不知道 就算是我们相信 说好有时光机 我谢谢你 MUSIC 妖兽扰乱人间秩序 血腥如浪潮般来袭 我小
prefix-不分开:  不分开 没有爱过  过去 你流眼泪 分手说不出来  分手说不出来 海鸟跟鱼相爱 只是一场意外 我

31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 3.9832, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开不知道 就算是我不懂 能不能原谅我 请不要把分手当作你的请求 我知道坚持要走是你受伤的借口 请你回头
prefix-不分开:  不分开不知道 简单一句爱说不出来 经过山丘 嗨呦嗨呦 汗一直流 行李不多 但思念一定带走 朗gia刚掉路 


Epoch [226/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 3.9633, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开不知道 你会更难过去的熟悉 照后镜的你比脑海清晰 你我距离就像打滑和那飘移 差狠远但看似狠接近 车灯
prefix-不分开:  不分开不知道 力没有伤害 你 靠着我的肩膀 你 在我胸口睡著 像这样的生活 我爱你 你爱我 我想就这样牵着


Epoch [227/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 3.9437, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开不知道 你说你好累 已无法再爱上谁 风在山路吹 过往的画面 全都是我不对 细数惭愧我伤你几回 后视镜
prefix-不分开:  不分开 没办法 我不配 天空这个路  我说 不要再这样打我妈妈 我说的话你甘会听 不要再这样打我妈妈 难道


Epoch [228/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 3.9248, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开 没有了雨 雨刷开始不停左右 就像回忆 开始对我挥了挥了手 仪表板转动 在猜我会不会懂 速度再快也追
prefix-不分开:  不分开 知道很简单　 你说依赖是我们的阻碍 就算放开但能不能别没收我的爱　 当作我最后才明白 看